In [916]:
import numpy as np
import pandas as pd
from IPython.display import display
from sklearn import model_selection, preprocessing, neighbors
import seaborn as sns
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from haversine import haversine

import matplotlib.pyplot as plt
import seaborn as sn
import missingno as msno
%matplotlib inline

In [404]:
###
### Combine 2016 and 2017 data
###
#properties2016 = pd.read_csv('properties_2016.csv')
#print("2016 Properties - {} rows with {} features".format(*properties2016.shape))
#properties2017 = pd.read_csv('properties_2017.csv')
#print("2017 Properties - {} rows with {} features".format(*properties2017.shape))
#train2016 = pd.read_csv('train_2016_v2.csv', parse_dates=['transactiondate'])
#print("2016 Train - {} rows with {} features".format(*train2016.shape))
#train2017 = pd.read_csv('train_2017.csv', parse_dates=['transactiondate'])
#print("2017 Train - {} rows with {} features".format(*train2017.shape))
#train = pd.merge(train2016, properties2016,how='left',on='parcelid')
#train2 = pd.merge(train2017, properties2017,how='left',on='parcelid')
#train = train.append(train2)
#
#msk = np.random.rand(len(train)) <= 0.8
#
#final_train = train[msk]
#final_train.to_csv('train.csv', index=False)
#final_test = train[~msk]
#final_test.to_csv('test.csv', index=False)

134627 rows with 60 features


## Dealing with missing data



In [373]:
###
### edits to living area data
###


print("finishedfloor1squarefeet null count: ", train.finishedfloor1squarefeet.isnull().sum())
#index = train.finishedfloor1squarefeet.isnull()
#train.loc[index,'finishedfloor1squarefeet'] = 0
print("finishedfloor1squarefeet:")
print(train.finishedfloor1squarefeet.value_counts())
train.drop(['finishedfloor1squarefeet'], axis=1, inplace=True) # Lots of missing data


print("finishedsquarefeet6 null count: ", train.finishedsquarefeet6.isnull().sum())
print("finishedsquarefeet6:")
print(train.finishedsquarefeet6.value_counts())
train.drop(['finishedsquarefeet6'], axis=1, inplace=True) # Lots of missing data


print("finishedsquarefeet13 null count: ", train.finishedsquarefeet13.isnull().sum())
print("finishedsquarefeet13:")
print(train.finishedsquarefeet13.value_counts())
train.drop(['finishedsquarefeet13'], axis=1, inplace=True) # Lots of missing data


print("finishedsquarefeet15 null count: ", train.finishedsquarefeet15.isnull().sum())
print("finishedsquarefeet15:")
print(train.finishedsquarefeet15.value_counts())
train.drop(['finishedsquarefeet15'], axis=1, inplace=True) # Lots of missing data


print("finishedsquarefeet50 null count: ", train.finishedsquarefeet50.isnull().sum())
print("finishedsquarefeet50:")
print(train.finishedsquarefeet50.value_counts())
train.drop(['finishedsquarefeet50'], axis=1, inplace=True) # same as finishedfloor1squarefeet







finishedfloor1squarefeet null count:  31128
finishedfloor1squarefeet:
1431.0    12
1226.0    11
713.0     11
734.0     10
1040.0    10
1252.0    10
1043.0     9
1612.0     9
1462.0     9
1016.0     9
1064.0     9
1058.0     8
1233.0     8
1110.0     8
1891.0     8
856.0      8
1500.0     8
1123.0     8
1498.0     8
1205.0     7
1290.0     7
1368.0     7
1227.0     7
979.0      7
780.0      7
798.0      7
1155.0     7
912.0      7
2154.0     7
1547.0     6
          ..
493.0      1
1635.0     1
2516.0     1
1676.0     1
1663.0     1
3502.0     1
746.0      1
740.0      1
778.0      1
818.0      1
852.0      1
2222.0     1
1732.0     1
2461.0     1
2402.0     1
792.0      1
3157.0     1
1909.0     1
1788.0     1
476.0      1
3447.0     1
1283.0     1
1532.0     1
1823.0     1
1318.0     1
894.0      1
1332.0     1
1250.0     1
1481.0     1
1543.0     1
Name: finishedfloor1squarefeet, Length: 1272, dtype: int64
finishedsquarefeet6 null count:  33590
finishedsquarefeet6:
2172.0    2
2582.0

100%       8294
75%        8281
50%        8271
25%        8148
Unknown     199
Name: New_CalculatedFinishedSquareFeet, dtype: int64


50%        9444
75%        7926
100%       7901
25%        6260
Unknown    1662
Name: New_FinishedSquareFeet12, dtype: int64


In [850]:
train.loc[(train.unitcnt <= 0) | (train.unitcnt.isnull()), 'New_UnitCount'] = 'Unknown'
train.loc[train.unitcnt == 1, 'New_UnitCount'] = 'Single'
train.loc[train.unitcnt > 1, 'New_UnitCount'] = 'Multiple'


print(train.New_UnitCount.value_counts())


Single      20258
Unknown     11620
Multiple     1383
Name: New_UnitCount, dtype: int64


In [851]:

train.loc[(train.buildingqualitytypeid.isnull()) | (train.buildingqualitytypeid <= 0), 'New_BuildingQuality'] = 'Unknown'
train.loc[(train.buildingqualitytypeid >= 1) & (train.buildingqualitytypeid <= 4), 'New_BuildingQuality'] = 'Good'
train.loc[(train.buildingqualitytypeid >= 5) & (train.buildingqualitytypeid <= 8), 'New_BuildingQuality'] = 'OK'
train.loc[train.buildingqualitytypeid > 8, 'New_BuildingQuality'] = 'Bad'

print(train.New_BuildingQuality.value_counts())


OK         13089
Unknown    11987
Good        7205
Bad          980
Name: New_BuildingQuality, dtype: int64


In [852]:

train.loc[(train.calculatedbathnbr.isnull()) | (train.calculatedbathnbr <= 0), 'New_CalculatedBath'] = 'Unknown'
train.loc[(train.calculatedbathnbr >= 1) & (train.calculatedbathnbr <= 4), 'New_CalculatedBath'] = 'Few'
train.loc[train.calculatedbathnbr >= 4, 'New_CalculatedBath'] = 'Many'

print(train.New_CalculatedBath.value_counts())

Few        30420
Many        2477
Unknown      364
Name: New_CalculatedBath, dtype: int64


In [853]:
print("fullbathcnt null count: ", train.fullbathcnt.isnull().sum())
index = train.fullbathcnt.isnull()


train.loc[(train.fullbathcnt.isnull()) | (train.fullbathcnt <= 0), 'New_FullBath'] = 'Unknown'
train.loc[(train.fullbathcnt >= 1) & (train.fullbathcnt < 4), 'New_FullBath'] = 'Few'
train.loc[train.fullbathcnt >= 4, 'New_FullBath'] = 'Many'


print(train.New_FullBath.value_counts())
#train.drop(['fullbathcnt'], axis=1, inplace=True)

fullbathcnt null count:  364
Few        30430
Many        2467
Unknown      364
Name: New_FullBath, dtype: int64


# Important Features

In [892]:
ulimit90 = np.nanpercentile(train.calculatedfinishedsquarefeet.values, 90.)
ulimit95 = np.nanpercentile(train.calculatedfinishedsquarefeet.values, 95.)
ulimit99 = np.nanpercentile(train.calculatedfinishedsquarefeet.values, 99.)
ulimit995 = np.nanpercentile(train.calculatedfinishedsquarefeet.values, 99.5)
llimit10 = np.nanpercentile(train.calculatedfinishedsquarefeet.values, 10.)
llimit5 = np.nanpercentile(train.calculatedfinishedsquarefeet.values, 5.)
llimit1 = np.nanpercentile(train.calculatedfinishedsquarefeet.values, 1.)
llimit05 = np.nanpercentile(train.calculatedfinishedsquarefeet.values, 0.5)

train.loc[(train.calculatedfinishedsquarefeet >= ulimit90), 'New_CalcFinSqFt'] = '90%'
train.loc[(train.calculatedfinishedsquarefeet >= ulimit95), 'New_CalcFinSqFt'] = '95%'
train.loc[(train.calculatedfinishedsquarefeet >= ulimit99), 'New_CalcFinSqFt'] = '99%'
train.loc[(train.calculatedfinishedsquarefeet >= ulimit995), 'New_CalcFinSqFt'] = '99.5%'
train.loc[(train.calculatedfinishedsquarefeet <= llimit10), 'New_CalcFinSqFt'] = '10%'
train.loc[(train.calculatedfinishedsquarefeet <= llimit5), 'New_CalcFinSqFt'] = '5%'
train.loc[(train.calculatedfinishedsquarefeet <= llimit1), 'New_CalcFinSqFt'] = '1%'
train.loc[(train.calculatedfinishedsquarefeet <= llimit05), 'New_CalcFinSqFt'] = '0.5%'

print(train.New_CalcFinSqFt.value_counts())


train.loc[(train.calculatedfinishedsquarefeet <= 0) | (train.calculatedfinishedsquarefeet.isnull()), 'New_CalculatedFinishedSquareFeet'] = 'Unknown'
train.loc[(train['calculatedfinishedsquarefeet'] > 0) & (train['calculatedfinishedsquarefeet'] < 1180), 'New_CalculatedFinishedSquareFeet'] = '25%'
train.loc[(train['calculatedfinishedsquarefeet'] > 1180) & (train['calculatedfinishedsquarefeet'] < 1540), 'New_CalculatedFinishedSquareFeet'] = '50%'
train.loc[(train['calculatedfinishedsquarefeet'] > 1540) & (train['calculatedfinishedsquarefeet'] < 2100), 'New_CalculatedFinishedSquareFeet'] = '75%'
train.loc[train.calculatedfinishedsquarefeet >= 2100, 'New_CalculatedFinishedSquareFeet'] = '100%'
print(train.New_CalculatedFinishedSquareFeet.value_counts())

10%      6712
90%      6685
95%      5371
5%       5356
0.5%      676
1%        672
99.5%     671
99%       670
Name: New_CalcFinSqFt, dtype: int64
100%       33611
50%        33491
75%        33319
25%        33234
Unknown      697
Name: New_CalculatedFinishedSquareFeet, dtype: int64


In [881]:
ulimit90 = np.nanpercentile(train.finishedsquarefeet12.values, 90)
ulimit95 = np.nanpercentile(train.finishedsquarefeet12.values, 95)
ulimit99 = np.nanpercentile(train.finishedsquarefeet12.values, 99)
ulimit995 = np.nanpercentile(train.finishedsquarefeet12.values, 99.5)
llimit10 = np.nanpercentile(train.finishedsquarefeet12.values, 10)
llimit5 = np.nanpercentile(train.finishedsquarefeet12.values, 5)
llimit1 = np.nanpercentile(train.finishedsquarefeet12.values, 1)
llimit05 = np.nanpercentile(train.finishedsquarefeet12.values, .5)

train.loc[(train.finishedsquarefeet12 >= ulimit90), 'New_FinSqFt12'] = '90%'
train.loc[(train.finishedsquarefeet12 >= ulimit95), 'New_FinSqFt12'] = '95%'
train.loc[(train.finishedsquarefeet12 >= ulimit99), 'New_FinSqFt12'] = '99%'
train.loc[(train.finishedsquarefeet12 >= ulimit995), 'New_FinSqFt12'] = '99.5%'
train.loc[(train.finishedsquarefeet12 <= llimit10), 'New_FinSqFt12'] = '10%'
train.loc[(train.finishedsquarefeet12 <= llimit5), 'New_FinSqFt12'] = '5%'
train.loc[(train.finishedsquarefeet12 <= llimit1), 'New_FinSqFt12'] = '1%'
train.loc[(train.finishedsquarefeet12 <= llimit05), 'New_FinSqFt12'] = '0.5%'

print(train.New_FinSqFt12.value_counts())


#train.loc[(train.finishedsquarefeet12 <= 0) | (train.finishedsquarefeet12.isnull()), 'New_FinishedSquareFeet12'] = 'Unknown'
#train.loc[(train['finishedsquarefeet12'] > 0) & (train['finishedsquarefeet12'] < 1100), 'New_FinishedSquareFeet12'] = '25%'
#train.loc[(train['finishedsquarefeet12'] > 1100) & (train['finishedsquarefeet12'] < 1520), 'New_FinishedSquareFeet12'] = '50%'
#train.loc[(train['finishedsquarefeet12'] > 1520) & (train['finishedsquarefeet12'] < 2065), 'New_FinishedSquareFeet12'] = '75%'
#train.loc[train.finishedsquarefeet12 >= 2065, 'New_FinishedSquareFeet12'] = '100%'
#
#print(train.New_FinishedSquareFeet12.value_counts())

90%      1588
10%      1586
5%       1272
95%      1264
0.5%      160
99.5%     158
1%        158
99%       158
Name: New_FinSqFt12, dtype: int64


In [882]:
ulimit90 = np.nanpercentile(train.lotsizesquarefeet.values, 90)
ulimit95 = np.nanpercentile(train.lotsizesquarefeet.values, 95)
ulimit99 = np.nanpercentile(train.lotsizesquarefeet.values, 99)
ulimit995 = np.nanpercentile(train.lotsizesquarefeet.values, 99.5)
llimit10 = np.nanpercentile(train.lotsizesquarefeet.values, 10)
llimit5 = np.nanpercentile(train.lotsizesquarefeet.values, 5)
llimit1 = np.nanpercentile(train.lotsizesquarefeet.values, 1)
llimit05 = np.nanpercentile(train.lotsizesquarefeet.values, .5)

train.loc[(train.lotsizesquarefeet >= ulimit90), 'New_LotSizeSqFt'] = '90%'
train.loc[(train.lotsizesquarefeet >= ulimit95), 'New_LotSizeSqFt'] = '95%'
train.loc[(train.lotsizesquarefeet >= ulimit99), 'New_LotSizeSqFt'] = '99%'
train.loc[(train.lotsizesquarefeet >= ulimit995), 'New_LotSizeSqFt'] = '99.5%'
train.loc[(train.lotsizesquarefeet <= llimit10), 'New_LotSizeSqFt'] = '10%'
train.loc[(train.lotsizesquarefeet <= llimit5), 'New_LotSizeSqFt'] = '5%'
train.loc[(train.lotsizesquarefeet <= llimit1), 'New_LotSizeSqFt'] = '1%'
train.loc[(train.lotsizesquarefeet <= llimit05), 'New_LotSizeSqFt'] = '0.5%'

print(train.New_LotSizeSqFt.value_counts())

#train.loc[train.lotsizesquarefeet.isnull(), 'New_LotSizeSquareFeet'] = 'Unknown'
#train.loc[(train.lotsizesquarefeet > 0) & (train.lotsizesquarefeet <= 5700), 'New_LotSizeSquareFeet'] = '25%'
#train.loc[(train.lotsizesquarefeet > 5700) & (train.lotsizesquarefeet <= 7200), 'New_LotSizeSquareFeet'] = '50%'
#train.loc[(train.lotsizesquarefeet > 7200) & (train.lotsizesquarefeet <= 11750), 'New_LotSizeSquareFeet'] = '75%'
#train.loc[(train.lotsizesquarefeet > 11750), 'New_LotSizeSquareFeet'] = '100%'
#
#print(train.New_LotSizeSquareFeet.value_counts())

90%      1477
10%      1473
5%       1188
95%      1184
99.5%     149
0.5%      148
1%        148
99%       147
Name: New_LotSizeSqFt, dtype: int64


In [883]:
ulimit90 = np.nanpercentile(train.structuretaxvaluedollarcnt.values, 90)
ulimit95 = np.nanpercentile(train.structuretaxvaluedollarcnt.values, 95)
ulimit99 = np.nanpercentile(train.structuretaxvaluedollarcnt.values, 99)
ulimit995 = np.nanpercentile(train.structuretaxvaluedollarcnt.values, 99.5)
llimit10 = np.nanpercentile(train.structuretaxvaluedollarcnt.values, 10)
llimit5 = np.nanpercentile(train.structuretaxvaluedollarcnt.values, 5)
llimit1 = np.nanpercentile(train.structuretaxvaluedollarcnt.values, 1)
llimit05 = np.nanpercentile(train.structuretaxvaluedollarcnt.values, .5)

train.loc[(train.structuretaxvaluedollarcnt >= ulimit90), 'New_StructTaxValueDollarCnt'] = '90%'
train.loc[(train.structuretaxvaluedollarcnt >= ulimit95), 'New_StructTaxValueDollarCnt'] = '95%'
train.loc[(train.structuretaxvaluedollarcnt >= ulimit99), 'New_StructTaxValueDollarCnt'] = '99%'
train.loc[(train.structuretaxvaluedollarcnt >= ulimit995), 'New_StructTaxValueDollarCnt'] = '99.5%'
train.loc[(train.structuretaxvaluedollarcnt <= llimit10), 'New_StructTaxValueDollarCnt'] = '10%'
train.loc[(train.structuretaxvaluedollarcnt <= llimit5), 'New_StructTaxValueDollarCnt'] = '5%'
train.loc[(train.structuretaxvaluedollarcnt <= llimit1), 'New_StructTaxValueDollarCnt'] = '1%'
train.loc[(train.structuretaxvaluedollarcnt <= llimit05), 'New_StructTaxValueDollarCnt'] = '0.5%'

print(train.New_StructTaxValueDollarCnt.value_counts())

#train.loc[train.structuretaxvaluedollarcnt.isnull(), 'New_StructureTaxValue'] = 'Unknown'
#train.loc[(train.structuretaxvaluedollarcnt > 0) & (train.structuretaxvaluedollarcnt <= 100000), 'New_StructureTaxValue'] = 'Under100'
#train.loc[(train.structuretaxvaluedollarcnt > 100000) & (train.structuretaxvaluedollarcnt <= 200000), 'New_StructureTaxValue'] = 'Under200'
#train.loc[(train.structuretaxvaluedollarcnt > 200000) & (train.structuretaxvaluedollarcnt <= 300000), 'New_StructureTaxValue'] = 'Under300'
#train.loc[(train.structuretaxvaluedollarcnt > 300000) & (train.structuretaxvaluedollarcnt <= 400000), 'New_StructureTaxValue'] = 'Under400'
#train.loc[(train.structuretaxvaluedollarcnt > 400000) & (train.structuretaxvaluedollarcnt <= 600000), 'New_StructureTaxValue'] = 'Under600'
#train.loc[(train.structuretaxvaluedollarcnt > 600000) & (train.structuretaxvaluedollarcnt <= 800000), 'New_StructureTaxValue'] = 'Under800'
#train.loc[(train.structuretaxvaluedollarcnt > 800000) & (train.structuretaxvaluedollarcnt <= 1000000), 'New_StructureTaxValue'] = 'Under1Mil'
#train.loc[train.structuretaxvaluedollarcnt > 1000000, 'New_StructureTaxValue'] = 'Over1Mil'
#
#print(train.New_StructureTaxValue.value_counts())

90%      1658
10%      1658
5%       1326
95%      1326
0.5%      166
99.5%     166
1%        166
99%       166
Name: New_StructTaxValueDollarCnt, dtype: int64


In [884]:
ulimit90 = np.nanpercentile(train.taxamount.values, 90)
ulimit95 = np.nanpercentile(train.taxamount.values, 95)
ulimit99 = np.nanpercentile(train.taxamount.values, 99)
ulimit995 = np.nanpercentile(train.taxamount.values, 99.5)
llimit10 = np.nanpercentile(train.taxamount.values, 10)
llimit5 = np.nanpercentile(train.taxamount.values, 5)
llimit1 = np.nanpercentile(train.taxamount.values, 1)
llimit05 = np.nanpercentile(train.taxamount.values, .5)

train.loc[(train.taxamount >= ulimit90), 'New_TaxAmount'] = '90%'
train.loc[(train.taxamount >= ulimit95), 'New_TaxAmount'] = '95%'
train.loc[(train.taxamount >= ulimit99), 'New_TaxAmount'] = '99%'
train.loc[(train.taxamount >= ulimit995), 'New_TaxAmount'] = '99.5%'
train.loc[(train.taxamount <= llimit10), 'New_TaxAmount'] = '10%'
train.loc[(train.taxamount <= llimit5), 'New_TaxAmount'] = '5%'
train.loc[(train.taxamount <= llimit1), 'New_TaxAmount'] = '1%'
train.loc[(train.taxamount <= llimit05), 'New_TaxAmount'] = '0.5%'

print(train.New_TaxAmount.value_counts())


#train.loc[train.taxamount.isnull(), 'New_TaxAmount'] = 'Unknown'
#train.loc[(train.taxamount > 0) & (train.taxamount <= 3000), 'New_TaxAmount'] = 'Under3k'
#train.loc[(train.taxamount > 3000) & (train.taxamount <= 5000), 'New_TaxAmount'] = 'Under5k'
#train.loc[(train.taxamount > 5000) & (train.taxamount <= 7000), 'New_TaxAmount'] = 'Under7k'
#train.loc[(train.taxamount > 7000) & (train.taxamount <= 10000), 'New_TaxAmount'] = 'Under10k'
#train.loc[(train.taxamount > 10000) & (train.taxamount <= 50000), 'New_TaxAmount'] = 'Under50k'
#train.loc[train.taxamount > 50000, 'New_TaxAmount'] = 'Over50k'
#
#print(train.New_TaxAmount.value_counts())

90%      1662
10%      1662
5%       1330
95%      1330
0.5%      167
99.5%     167
1%        166
99%       166
Name: New_TaxAmount, dtype: int64



# Preprocess Features

In [957]:
train = pd.read_csv("test.csv", parse_dates=['transactiondate'])
print("{} rows with {} features".format(*train.shape))

33261 rows with 60 features


/Users/BurrusRA/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (51) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Mod 1

In [958]:

train['airconditioningtypeid'] = train.airconditioningtypeid.replace({1 : 'Central', 2 : 'ChilledWater', 3 : 'EvaporativeCooler', 4 : 'GeoThermal', 5 : 'None', 6 : 'Other', 7 : 'PackagedACUnit', 8 : 'Partial', 9 : 'Refrigeration', 10 : 'Ventilation', 11 : 'WallUnit', 12 : 'WindowUnit', 13 : 'Yes'})

train['heatingorsystemtypeid'] = train.heatingorsystemtypeid.replace({1 : 'Baseboard', 2 : 'Central', 3 : 'Coal', 4 : 'Convection', 5 : 'Electric', 6 : 'ForcedAir', 7 : 'FloorWall', 8 : 'Gas', 9 : 'GeoThermal', 10 : 'Gravity', 11 : 'HeatPump', 12 : 'HotWater', 13 : 'None',
                                                                     14 : 'Other', 15 : 'Oil', 16 : 'Partial', 17 : 'Propane', 18 : 'Radiant', 19: 'Steam', 20 : 'Solar', 21 : 'SpaceSuspended', 22 : 'Vent', 23 : 'WoodBurning', 24 : 'Yes', 25 : 'Zone'})

train['propertylandusetypeid'] = train.propertylandusetypeid.replace({31 : 'CommercialOfficeResidentialMixed', 46 : 'multistorystore', 246 : 'Duplex', 247 : 'triplex', 248 : 'Quadruplex', 260 : 'ResidentialGeneral', 261 : 'SingleFam', 262 : 'Rural', 263 : 'MobileHome', 264 : 'Townhouse', 265 : 'Cluster', 266 : 'Condo', 267 : 'Coop',
                                                                     268 : 'RowHouse', 269 : 'PannedUnit', 270 : 'ResCommon', 271 : 'TimeShare', 273 : 'Bungalow', 274: 'ZeroLotLine', 275 : 'PrefabHome', 276 : 'PatioHome', 279 : 'IngerredSingleFam', 290 : 'Vacant', 291 : 'ResidentialVacant'})


##
## these id fields have >97% of data missing
##

train['storytypeid'] = train.storytypeid.replace({7 : 'Basement'})

train['architecturalstyletypeid'] = train.architecturalstyletypeid.replace({2 : 'Bungalow', 3 : 'CapeCod', 7 : 'Contemporary', 8 : 'Conventional', 10 : 'FrenchProvincial', 21 : 'Ranch'})

train['typeconstructiontypeid'] = train.typeconstructiontypeid.replace({4 : 'Concrete', 6 : 'Frame', 10 : 'Metal', 13 : 'Masonry'})

train['buildingclasstypeid'] = train.buildingclasstypeid.replace({3 : 'NonCombust', 4 : 'WoodFrames'})




In [959]:
#columns = ['propertycountylandusecode', 'propertylandusetypeid', 'regionidcity', 'regionidcounty', 'regionidneighborhood', 'regionidzip', 'assessmentyear', 'taxdelinquencyyear']
columns_no_region = ['propertycountylandusecode', 'propertylandusetypeid', 'assessmentyear', 'taxdelinquencyyear']
for c in columns_no_region:
    train[c] = train[c].astype(object)
    
objects = train.select_dtypes(include=[object])
print(objects.columns.values)

['airconditioningtypeid' 'architecturalstyletypeid' 'buildingclasstypeid'
 'hashottuborspa' 'heatingorsystemtypeid' 'propertycountylandusecode'
 'propertylandusetypeid' 'propertyzoningdesc' 'storytypeid'
 'typeconstructiontypeid' 'fireplaceflag' 'assessmentyear'
 'taxdelinquencyflag' 'taxdelinquencyyear']


In [960]:
train.drop(['propertycountylandusecode', 'propertyzoningdesc'], axis=1, inplace=True)
train = pd.get_dummies(train, dummy_na=True)

# Mod 2

In [961]:
###
### Create new date fields
###
train["transaction_year"] = train["transactiondate"].dt.year
train["transaction_month"] = (train["transactiondate"].dt.year - 2016)*12 + train["transactiondate"].dt.month
train["transaction_day"] = train["transactiondate"].dt.day
train["transaction_quarter"] = (train["transactiondate"].dt.year - 2016)*4 +train["transactiondate"].dt.quarter
train.drop(["transactiondate"], inplace=True, axis=1)

# Mod 3

In [962]:
train["New_YearsSinceBuild"] = 2018 - train['yearbuilt']
#print(train.New_YearsSinceBuild.value_counts())
train.drop(["yearbuilt"], inplace=True, axis=1)

# Mod 5

In [963]:
train['New_FinishedLivingToLotSizeRatio'] = train.calculatedfinishedsquarefeet / train.lotsizesquarefeet

train['New_StructureTaxToLandTaxRatio'] = train.structuretaxvaluedollarcnt / train.landtaxvaluedollarcnt

train['New_TaxRatio'] = train.taxamount / train.taxvaluedollarcnt


In [964]:
#print(train.New_haversine.value_counts())
#print(train.latitude.value_counts())
#print(train.longitude.value_counts())

In [965]:
#Distance and angle from city center?
train.loc[train.latitude.isnull(), 'latitude'] = 0.
train.loc[train.longitude.isnull(), 'longitude'] = 0.

train.latitude /= 1000000.
train.longitude /= 1000000.

train.latitude = train.latitude.astype(float)
train.longitude = train.longitude.astype(float)

def get_haversine_la(row):
    return haversine((34.042037, -118.247142),(row['latitude'], row['longitude']))

def get_haversine_beverlyhills(row):
    return haversine((34.074692, -118.401294),(row['latitude'], row['longitude']))

def get_haversine_sanclemente(row):
    return haversine((33.430520, -117.618732),(row['latitude'], row['longitude']))



train['New_haversine_la'] = train.apply(get_haversine_la, axis=1)
train['New_haversine_beverlyhills'] = train.apply(get_haversine_beverlyhills, axis=1)
train['New_haversine_sanclemente'] = train.apply(get_haversine_sanclemente, axis=1)




# CatBoost

In [812]:
missing_df = train.isnull().sum(axis=0).reset_index()
missing_df.columns = ['column_name', 'missing_count']
missing_df['missing_ratio'] = missing_df['missing_count'] / train.shape[0]
high_miss_rate = missing_df['missing_ratio']>=0.97
print(missing_df[high_miss_rate])
print(missing_df[high_miss_rate].column_name.values)
###
### Fill in Nan
###
train.fillna(-999,inplace=True)

                 column_name  missing_count  missing_ratio
3   architecturalstyletypeid          33176       0.997444
4               basementsqft          33236       0.999248
7        buildingclasstypeid          33255       0.999820
10                decktypeid          32988       0.991792
14      finishedsquarefeet13          33248       0.999609
17       finishedsquarefeet6          33100       0.995159
23            hashottuborspa          32528       0.977962
29               poolsizesum          32882       0.988605
30              pooltypeid10          32953       0.990740
31               pooltypeid2          32836       0.987222
42               storytypeid          33236       0.999248
44    typeconstructiontypeid          33167       0.997174
47        yardbuildingsqft26          33221       0.998797
49             fireplaceflag          33194       0.997986
55        taxdelinquencyflag          32343       0.972400
56        taxdelinquencyyear          32343       0.9724

In [813]:
train.drop(['architecturalstyletypeid', 'basementsqft', 'buildingclasstypeid',
 'decktypeid', 'finishedsquarefeet13', 'finishedsquarefeet6', 'hashottuborspa',
 'poolsizesum', 'pooltypeid10', 'pooltypeid2', 'storytypeid',
 'typeconstructiontypeid', 'yardbuildingsqft17', 'yardbuildingsqft26',
 'fireplaceflag', 'taxdelinquencyflag', 'taxdelinquencyyear'], inplace=True, axis=1)

In [814]:
train.drop(['propertycountylandusecode', 'propertyzoningdesc'], axis=1, inplace=True)

# Save data

In [966]:
filename = 'test_mod5.csv'
train.to_csv(filename,index=False)


### Remove fields and fill in any remaining missing data


In [906]:
print(list(train.columns.values))
print(len(train.columns.values))

['parcelid', 'logerror', 'basementsqft', 'bathroomcnt', 'bedroomcnt', 'buildingqualitytypeid', 'calculatedbathnbr', 'decktypeid', 'finishedfloor1squarefeet', 'calculatedfinishedsquarefeet', 'finishedsquarefeet12', 'finishedsquarefeet13', 'finishedsquarefeet15', 'finishedsquarefeet50', 'finishedsquarefeet6', 'fips', 'fireplacecnt', 'fullbathcnt', 'garagecarcnt', 'garagetotalsqft', 'latitude', 'longitude', 'lotsizesquarefeet', 'poolcnt', 'poolsizesum', 'pooltypeid10', 'pooltypeid2', 'pooltypeid7', 'rawcensustractandblock', 'regionidcity', 'regionidcounty', 'regionidneighborhood', 'regionidzip', 'roomcnt', 'threequarterbathnbr', 'unitcnt', 'yardbuildingsqft17', 'yardbuildingsqft26', 'numberofstories', 'structuretaxvaluedollarcnt', 'taxvaluedollarcnt', 'landtaxvaluedollarcnt', 'taxamount', 'censustractandblock', 'New_FinishedLivingToLotSizeRatio', 'New_StructureTaxToLandTaxRatio', 'New_TaxRatio', 'airconditioningtypeid_Central', 'airconditioningtypeid_EvaporativeCooler', 'airconditioningty

In [905]:
###
### Handle object fields. Either remove, one-hot-encode, or convert to different dtype
###
objects = train.select_dtypes(include=[object])
print(objects.columns.values)

[]


In [896]:
objects = train.select_dtypes(include=[np.float64])

print(objects.columns.values)

['logerror' 'airconditioningtypeid' 'architecturalstyletypeid'
 'basementsqft' 'bathroomcnt' 'bedroomcnt' 'buildingclasstypeid'
 'buildingqualitytypeid' 'calculatedbathnbr' 'decktypeid'
 'finishedfloor1squarefeet' 'calculatedfinishedsquarefeet'
 'finishedsquarefeet12' 'finishedsquarefeet13' 'finishedsquarefeet15'
 'finishedsquarefeet50' 'finishedsquarefeet6' 'fips' 'fireplacecnt'
 'fullbathcnt' 'garagecarcnt' 'garagetotalsqft' 'heatingorsystemtypeid'
 'latitude' 'longitude' 'lotsizesquarefeet' 'poolcnt' 'poolsizesum'
 'pooltypeid10' 'pooltypeid2' 'pooltypeid7' 'propertylandusetypeid'
 'rawcensustractandblock' 'regionidcity' 'regionidcounty'
 'regionidneighborhood' 'regionidzip' 'roomcnt' 'storytypeid'
 'threequarterbathnbr' 'typeconstructiontypeid' 'unitcnt'
 'yardbuildingsqft17' 'yardbuildingsqft26' 'yearbuilt' 'numberofstories'
 'structuretaxvaluedollarcnt' 'taxvaluedollarcnt' 'assessmentyear'
 'landtaxvaluedollarcnt' 'taxamount' 'taxdelinquencyyear'
 'censustractandblock']


In [895]:
#PCA 

from sklearn.decomposition import PCA

# TODO: Apply PCA by fitting the good data with the same number of dimensions as features

pca = PCA(n_components=20).fit(train)

# TODO: Transform log_samples using the PCA fit above
#pca_samples = pca.transform(log_samples)

# Generate PCA results plot
pca_results = vs.pca_results(train, pca)

print( np.cumsum(pca_results['Explained Variance']))

ValueError: could not convert string to float: 'Y'